In [1]:
import nltk   
from bs4 import BeautifulSoup
from collections import Counter
import magic
import sys 
import tqdm
import time
from nltk.corpus import stopwords
import re
from os import listdir
from os.path import isfile, join
import numpy as np
import heapq
from collections import defaultdict

In [2]:
patt = re.compile (r'[\W^\d]+', flags=re.UNICODE)

In [3]:
def read_urls (f_name):
    urls = {}
    with open(f_name) as f:
        content = f.readlines()
    content = [x.strip() for x in content]
    for url in content:
        split = url.split ('\t')
        urls[int(split[0])] = split[1].decode ('utf-8')
    return urls

In [26]:
def read_queries (f_name):
    urls = {}
    patt = re.compile (r'[\W^\d]+', flags=re.UNICODE)
    with open(f_name) as f:
        content = f.readlines()
    content = [x.strip() for x in content]
    for url in content:
        split = url.split ('\t')
        urls[int(split[0])] = set ([word for word in set (patt.split (split[1].decode ('utf-8').lower ())) if word not in stopwords.words('russian')])
        if u'' in urls[int(split[0])]:
            urls[int(split[0])].remove (u'')
    return urls

In [27]:
urls = read_urls ("urls.numerate.txt")

In [28]:
queries = read_queries ("queries.numerate.txt")

In [7]:
def read_doc (doc_name):
    with open(doc_name) as f:
        name = f.readline ()
        html = f.read()
        m = magic.Magic(mime_encoding=True)
        encoding = m.from_buffer(html)
        try:
            html = html.decode (encoding)
        except:
#             sys.stderr.write ("{}, unknown encoding. skipping".format (doc_name))
            return None
            
    soup = BeautifulSoup(html)

    # kill all script and style elements
    for script in soup(["script", "style"]):
        script.extract()    # rip it out

    # get text
    text = soup.get_text()
    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    text = '\n'.join(chunk for chunk in chunks if chunk)
    return name, text.lower ()

In [8]:
def count_idf (queries):
    mypath = 'content/'
    dirs = listdir(mypath)
    onlyfiles = []
    len_of_file = []
    for d in dirs:
        onlyfiles.extend ([mypath + d + '/' + f for f in listdir(mypath + d) ])
        print onlyfiles
    docs_for_querie = defaultdict (set)
    t = time.time ()
    words = Counter ()
    num = len (onlyfiles)
    skipped = []
    s_num = 0
    for i, f in enumerate(onlyfiles):
        doc = read_doc (f)
        if doc is None:
            skipped.append (f)
            s_num += 1
            continue
        word_in_doc = patt.split (doc[1])
        filtered_words = [word for word in set (word_in_doc) if word not in stopwords.words('russian')]
        len_of_file.append (len (word_in_doc))

        words.update (filtered_words)
#         for q in queries.iteritems ():
#             if q[1].issubset(filtered_words):
#                 docs_for_querie[q[0]].add (f)
        sys.stdout.write ("\r {}% scanned. {} skipped.                  ".format (round (i * 100. / num,3), s_num))
    sys.stdout.write ("\r Done. {} skipped. {} Time Elapsed.            ".format (s_num, time.time () - t))
    avrg_len = np.mean (len_of_file)
    return words, skipped, docs_for_querie, avrg_len, len (onlyfiles) - len (skipped)

In [9]:
words, skipped, doc_f_q, avrg_len, num_of_docs = count_idf (queries)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

/Users/mikhailbelozerov/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 174 of the file /Users/mikhailbelozerov/anaconda2/lib/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


 Done. 873 skipped. 39596.914866 Time Elapsed.            

In [10]:
words.pop (u'')

15880

In [11]:
def update_idf (words, num_of_docs):
    idf = {}
    for d in words.iteritems ():
        res = np.log ((num_of_docs - d[1] + 0.5) / (d[1] + 0.5))
        if res > 0:
            idf[d[0]] = res
        else:
            idf[d[0]] = 0
    return idf

In [12]:
def get_bm25 (query, idf, doc, k, b, avrg_len):
    words = Counter (doc)
    bm25 = 0
    len_D = len (doc)
    for q in query:
        bm25 += idf[q] * idf[q] * (k + 1) / ( idf[q] + k * (1 - b + b*len_D / avrg_len))
    return bm25

In [30]:
def count_bm25 (queries, idf, avrg_len, b=0.75, k=2.0):
    mypath = 'content/'
    dirs = listdir(mypath)
    onlyfiles = []
    for d in dirs:
        onlyfiles.extend ([mypath + d + '/' + f for f in listdir(mypath + d) ])
    docs_for_querie = defaultdict (list)
    t = time.time ()
    num = len (onlyfiles)
    skipped = []
    s_num = 0
    for i, f in enumerate(onlyfiles):
        doc = read_doc (f)
        if doc is None:
            skipped.append (f)
            s_num += 1
            continue
        doc_name = doc[0]
        doc = patt.split (doc[1])
        filtered_words = [word for word in set (doc) if word not in stopwords.words('russian')]
        for q in queries.iteritems ():
            if q[1].issubset(filtered_words):
                bm = get_bm25 (q[1], idf, doc,k,b,avrg_len)
                if len (docs_for_querie[q[0]]) == 10:
                    heapq.heappushpop (docs_for_querie[q[0]], (bm, doc_name))
                else:
                    heapq.heappush (docs_for_querie[q[0]], (bm, doc_name))
        sys.stdout.write ("\r {}% scanned. {} skipped.                  ".format (round (i * 100. / num,3), s_num))

    sys.stdout.write ("\r Done. {} skipped. {} Time Elapsed.            ".format (s_num, time.time () - t))

    return docs_for_querie

In [17]:
idf = update_idf (words, num_of_docs)

In [31]:
docs = count_bm25 (queries, idf, avrg_len)

 13.696% scanned. 108 skipped.                  

KeyboardInterrupt: 